In [1]:
#연습!! https://finance.naver.com/ 에 접속하여 달러 환률 정보 확인!!

import urllib.request as req
from bs4 import BeautifulSoup

url='https://finance.naver.com/'

dollar=req.urlopen(url)
src=dollar.read().decode('euc-kr')
soup=BeautifulSoup(src,'html.parser')
price=soup.select_one('div.group1 >table.tbl_home>tbody>.down.bold>td')
print(price)
print('환율',price.string,'원')

<td>107.04</td>
환율 107.04 원


In [2]:
## https://finance.naver.com/ 에서 거래 상위 10개를 추출하여 출력하세요!
# 종목과 가격 => 대한전기 2140과 같은 식으로 출력

url='https://finance.naver.com/'
dollar=req.urlopen(url)
src=dollar.read().decode('euc-kr')
soup=BeautifulSoup(src,'html.parser')
dic={}

trade=soup.select('.tbl_home>#_topItems1>tr>th>a')
p=soup.select('.tbl_home>#_topItems1>tr>td')
price
l=[]
name=[trade[i].string for i in range(10)]
price=[p[i].string for i in range(0,30,3)]
for i in range(len(name)):
    dic[name[i]]=price[i]

dic

{'KODEX 200선물인버스2X': '2,750',
 '수산중공업': '4,305',
 '사조동아원': '2,355',
 'KH 건설': '1,190',
 '한탑': '4,405',
 '대한전선': '2,125',
 'KODEX 코스닥150선물인버스': '4,755',
 '삼성 인버스 2X WTI원유 선물 ETN': '120',
 '대주산업': '4,265',
 'KODEX 인버스': '4,550'}

In [89]:
# 링크에 있는 내용을 한꺼번에 처리하기

#재귀적으로 탐색하기 위해서 사용하는 함수 urljoin(base,"상대경로")함수
from urllib.parse import urljoin
##상대경로
base='http://example.com/html/a.html'
print(urljoin(base,'b.html'))
print(urljoin(base,'sub/c.html'))
print(urljoin(base,'../index.html'))
print(urljoin(base,'../css/home.css'))

##절대경로
print(urljoin(base, '/index.html')) 

print(urljoin(base,'http://otherExample.com/wiki')) #풀 주소 쓰면 덮어씀
print(urljoin(base, '//otherExample.com/test'))

http://example.com/html/b.html
http://example.com/html/sub/c.html
http://example.com/index.html
http://example.com/css/home.css
http://example.com/index.html
http://otherExample.com/wiki
http://otherExample.com/test


In [115]:
url='https://docs.python.org/3.10/library/index.html'
t=urlparse(url)
t.netloc    # docs.python.org
t.path      # /3.10/library/index.html

'/3.10/library/index.html'

In [ ]:
## 처리 순서
# 1. html 분석
# 2. 링크 추출
# 3. 각 링크 대상에 대해 다음 동작 구동함...
# # 3-1. 다운로드 파일인 경우 다운로드 ...
# # 3-2. 파일 html이라면, 1으로 돌아가서 다시 실행


In [10]:
# 모든 페이지를 한꺼번에 다운받는 프로그램

# 모듈
from bs4 import BeautifulSoup
from urllib.request import *
from urllib.parse import *
from os import makedirs
import os.path,time,re

# 처리한 내용이 파일인지 여부 확인을 위한 변수
proc_files={}

#  html 내부에 있는 링크를 추출하는 함수 
def enum_links(html,base):
    soup=BeautifulSoup(html,'html.parser')
    links=soup.select("link[rel='stylesheet']") #css
    links+=soup.select('a[href]')   #링크
    result=[]
    #href 속성을 추출하고, 링크를 절대경로로 변환하여 저장
    for a in links:
        href=a.attrs['href']
        url=urljoin(base,href)
        result.append(url)
    return result

#  파일을 다운로드하고 저장하는 함수
def download_file(url):
    o=urlparse(url)
    savepath="./"+o.netloc+o.path
    if re.search(r"/$",savepath):   #savepath의 마지막 "/"이라면
        savepath+="index.html"      #기본문서 처리
    savedir=os.path.dirname(savepath)
    # 모두 다운됐는지 여부 확인
    if os.path.exists(savepath):
        return savepath
    #다운받을 디렉터리(폴더) 생성
    if not os.path.exists(savedir):
        print("mkdir=",savedir)
        makedirs(savedir)
    #파일 다운받기
    try:
        print("download=",url)
        urlretrieve(url,savepath)
        time.sleep(1)   #잠시멈춤
        return savepath
    except:
        print("다운로드 실패 : ",url)
        return None


# html을 분석하고 다운 받는 함수 (위에 두개를 실행하는 함수)
def analyze_html(url,root_url):
    savepath=download_file(url)
    if savepath is None:
        return
    if savepath in proc_files:  # 이미 처리되었다면 실행하지 않음
        return
    proc_files[savepath]=True
    print("analyze_html",url)
    
    #링크 추출
    html=open(savepath,'r',encoding='utf8').read()
    links=enum_links(html,url)

    for link_url in links:
        #링크가 루트 이외 경로를 나타낸다면 무시
        if link_url.find(root_url)!=0:
            if not re.search(r".css$",link_url):
                continue
    #HTML이라면
        if re.search(r".(html|htm)$",link_url):
            #재귀적으로 html 검색
            analyze_html(link_url,root_url)
            continue
    # 기타파일이라면
    download_file(link_url)   

# 일반 파이썬 파일인 경우 사용.... 
# if __name__ == "__main__":
#     # url에 있는 모든 것 다운받기 
#     url = "https://docs.python.org/3.10/library/"
#     analyze_html(url, url)

url = "https://docs.python.org/3.10/library/"
analyze_html(url, url)

analyze_html https://docs.python.org/3.10/library/
analyze_html https://docs.python.org/3.10/library/intro.html
analyze_html https://docs.python.org/3.10/library/functions.html
analyze_html https://docs.python.org/3.10/library/constants.html
analyze_html https://docs.python.org/3.10/library/stdtypes.html
analyze_html https://docs.python.org/3.10/library/exceptions.html
analyze_html https://docs.python.org/3.10/library/text.html
analyze_html https://docs.python.org/3.10/library/string.html
analyze_html https://docs.python.org/3.10/library/re.html
analyze_html https://docs.python.org/3.10/library/difflib.html
analyze_html https://docs.python.org/3.10/library/textwrap.html
analyze_html https://docs.python.org/3.10/library/unicodedata.html
analyze_html https://docs.python.org/3.10/library/stringprep.html
analyze_html https://docs.python.org/3.10/library/readline.html
analyze_html https://docs.python.org/3.10/library/rlcompleter.html
analyze_html https://docs.python.org/3.10/library/binary.

In [ ]:
### requests 패키지를 이용한 접근..



In [ ]:
# <form name="frm"  id="frm"  action="#" method="post">
# 	<div class="login_left">		
# 	<fieldset>
# 		<legend>한빛출판네트워크 로그인</legend>
# 		<label class="i_label" for="login_id"><strong></strong>
# 			<input name="m_id" id="m_id" type="text" value="" class="i_text" placeholder="아이디" onkeydown="javascript:if(event.keyCode==13){login_proc(); return false;}">
# 		</label> 
# 		<label class="i_label" for="login_pw"><strong></strong>
# 			<input name="m_passwd"  id="m_passwd" type="password" value="" class="i_text" placeholder="비밀번호" onkeydown="javascript:if(event.keyCode==13){login_proc(); return false;}">
# 		</label>	
# 		<label>
# 			<input  type="button" name="login_btn"  id="login_btn" value="로그인" class="btn_login" >					
# 		</label>		
# 	</div>
# </form>

#파이썬으로 로그인 하기
import requests
from bs4 import BeautifulSoup
from urllib.request import urljoin

#아이디
user="djagkdbs"
passwd='dytjq1868'

#세션 시작
session=requests.session()
#로그인하기
login_info={
    "m_id":user,
    "m_passwd":passwd
}
url_login="https://www.hanbit.co.kr/member/login.html"
res=session.post(url_login,data=login_info)
res.raise_for_status()  #오류가 발생하면 예외 발생

# 마이페이지에 접근
url_mypage="https://www.hanbit.co.kr/myhanbit/myhanbit.html"
res=session.get(url_mypage)
res.raise_for_status()
print(res.text)
soup=BeautifulSoup(res.txt,"html.parser")
mileage=soup.select_one(".mileage_section1 span")

In [ ]:
pip install requests

In [30]:
## requests모듈의 메서드
#get 요청
r=requests.get('http://www.naver.com')
r.content.decode('utf8')

#post 요청
formdata={"key1":"value1","key2":"value2"}
r=requests.post("http://example.com",data=formdata)
r.content

#업로드
r=requests.put("http://httpbin.org/put")
#삭제
r=requests.delete("http://httpbin.org/delete")
#헤더정보
r=requests.head('http://httpbin.org/get')
print(requests.head('http://www.naver.com').content)

b''


In [ ]:
from http.server import HTTPServer,SimpleHTTPRequestHandler

server=HTTPServer(('0.0.0.0',8100),SimpleHTTPRequestHandler)
server.serve_forever()

In [8]:
from selenium import webdriver
user='djagkdbs'
pw='dytjq1868'
path="c:\\webdriver\\chromedriver"
driver=webdriver.Chrome(path)


C:\Users\djagk\AppData\Local\Temp\ipykernel_7136\2521611793.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver=webdriver.Chrome(path)


In [9]:
# 한빛 로그인 페이지 
url_login="https://www.hanbit.co.kr/member/login.html"
driver.get(url_login)

In [10]:
# 로그인 박스에 id와 비밀번호 입력하기
id=driver.find_element_by_name('m_id')
id.clear()
id.send_keys(user)
passwd=driver.find_element_by_name('m_passwd')
passwd.clear()
passwd.send_keys(pw)

C:\Users\djagk\AppData\Local\Temp\ipykernel_7136\2044285165.py:2: DeprecationWarning: find_element_by_name is deprecated. Please use find_element(by=By.NAME, value=name) instead
  id=driver.find_element_by_name('m_id')
C:\Users\djagk\AppData\Local\Temp\ipykernel_7136\2044285165.py:5: DeprecationWarning: find_element_by_name is deprecated. Please use find_element(by=By.NAME, value=name) instead
  passwd=driver.find_element_by_name('m_passwd')


In [11]:
driver.find_element_by_class_name("btn_login").click()

C:\Users\djagk\AppData\Local\Temp\ipykernel_7136\131413140.py:1: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  driver.find_element_by_class_name("btn_login").click()


In [12]:
driver.find_element_by_class_name('myhanbit').click()

C:\Users\djagk\AppData\Local\Temp\ipykernel_7136\2176582922.py:1: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  driver.find_element_by_class_name('myhanbit').click()


In [19]:
mileage=driver.find_element_by_css_selector(".mileage_section1 span")
print("마일리지 : ",mileage.text)

마일리지 :  2,000


C:\Users\djagk\AppData\Local\Temp\ipykernel_7136\2124175063.py:1: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  mileage=driver.find_element_by_css_selector(".mileage_section1 span")


In [20]:
coin=driver.find_element_by_css_selector('.mileage_section2 span')
print("코인 : ",coin.text)

코인 :  0


C:\Users\djagk\AppData\Local\Temp\ipykernel_7136\2513211130.py:1: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  coin=driver.find_element_by_css_selector('.mileage_section2 span')


In [44]:

user='djagkdbs'
pw='dytjq7169'


C:\Users\djagk\AppData\Local\Temp\ipykernel_7136\3674593798.py:1: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  driver.find_element_by_class_name('link_login').click()


In [45]:
from random import random,randint
import time
driver.get('https://www.naver.com')
driver.find_element_by_class_name('link_login').click()

id=driver.find_element_by_id('id')
for i in user:
    t=random()
    time.sleep(t)
    id.send_keys(i)
time.sleep(1)
passwd=driver.find_element_by_id('pw')
for i in pw:
    t=random()
    time.sleep(t)
    passwd.send_keys(i)

form=driver.find_element_by_class_name("btn_login")
form.submit()

C:\Users\djagk\AppData\Local\Temp\ipykernel_7136\2091818011.py:4: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  driver.find_element_by_class_name('link_login').click()
C:\Users\djagk\AppData\Local\Temp\ipykernel_7136\2091818011.py:6: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  id=driver.find_element_by_id('id')
C:\Users\djagk\AppData\Local\Temp\ipykernel_7136\2091818011.py:12: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  passwd=driver.find_element_by_id('pw')
C:\Users\djagk\AppData\Local\Temp\ipykernel_7136\2091818011.py:18: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  form=driver.find_element_by_class_name("btn_login")


In [78]:
#Selenium을 통해서 DOM요소 추출
# 메서드
#여러개중에 처음 요소를 추출
# find_element_by_id(id)
# find_element_by_name(name)
# find_element_by_css_selector(query) : css선택자로 요소 하나 추출 
# find_element_by_xpath(query) :Xpath를 저장해 요소 하나 추출
# find_element_by_tag_name(name) 
# find_element_by_link_text(text) : 링크 텍스트로 요소 하나 추출
# find_element_by_partial_link_text(text): 링크 자식요소에 포함되어있는 요소로 하나 추출
# find_element_by_class_name(class)

# 모든 요소 추출
# find_elements_by_css_selector(query)
# find_elements_by_xpath(query)
# find_elements_by_tag_name(name)
# find_elements_by_class_name(class)
# find_elements_by_partial_link_text(text)

#NoSuchElementException : 찾고자하는 요소가 없는 경우에 발생
ulsel="#main_content>div>div._persist>div:nth-child(1)>div:nth-child(1)>div.cluster_body>ul>li"
test=driver.find_elements_by_css_selector(ulsel)
for i in test:
    print(i.text)
    print("-"*50)
ul="#main_content>div>div._persist>div:nth-child(1)>div:nth-child(1)>div.cluster_body>ul>li:nth-child(1)>div>a"
test2=driver.find_element_by_css_selector(ul)
print(test2.text)
ul="#main_content>div>div._persist>div:nth-child(1)>div:nth-child(1)>div.cluster_body>ul"
#>li:nth-child(1)>div.cluster_text>a
for i in range(1,len(test)+1):
    head=driver.find_element_by_css_selector(ul+f'>li:nth-child({i})>div.cluster_text>a')
    print(head.text)

C:\Users\djagk\AppData\Local\Temp\ipykernel_7136\1091021578.py:22: DeprecationWarning: find_elements_by_css_selector is deprecated. Please use find_elements(by=By.CSS_SELECTOR, value=css_selector) instead
  test=driver.find_elements_by_css_selector(ulsel)
C:\Users\djagk\AppData\Local\Temp\ipykernel_7136\1091021578.py:27: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  test2=driver.find_element_by_css_selector(ul)


文 사저 인근 보수단체 집회에 국민의힘도 비판… 시선 유도봉 설치되기도
李 “메시지 효과 주민들에 끼치는 불편 크기와 비례하지 않아” 문재인 전 대통령이 지난 10일 경남 양산의 평산마을에 도착해 주민과 지지자들을 향해 인사 …
세계일보 50
이상
+
--------------------------------------------------
뉴스1 30
이상
+
이낙연 "文 사저 앞 보수인사 소음시위 참담"…野, 집시법 개정 추진
--------------------------------------------------
JTBC 30
이상
+
정청래, 전직 대통령 사저 앞 과도한 시위 금지 법안 발의
--------------------------------------------------
국민일보 1,500
이상
+
“前대통령 사저 앞, 시위 금지”… 文지키기 나선 민주
--------------------------------------------------
文 사저 인근 보수단체 집회에 국민의힘도 비판… 시선 유도봉 설치되기도
文 사저 인근 보수단체 집회에 국민의힘도 비판… 시선 유도봉 설치되기도
이낙연 "文 사저 앞 보수인사 소음시위 참담"…野, 집시법 개정 추진
정청래, 전직 대통령 사저 앞 과도한 시위 금지 법안 발의
“前대통령 사저 앞, 시위 금지”… 文지키기 나선 민주


C:\Users\djagk\AppData\Local\Temp\ipykernel_7136\1091021578.py:32: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  head=driver.find_element_by_css_selector(ul+f'>li:nth-child({i})>div.cluster_text>a')


In [83]:
title=driver.execute_script('return document.title;')
title
alert=driver.execute_script("alert('test');")
alert
